# Plotting ORT Results

This narrative is not a core component of the ORT workflow, but is provided alongside it as we think it may be useful to you for vizualizing model results.

In [ ]:
# Import plotting libraries
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from kbpf_tools import speciessorter

## Designate Output Files

Here we list the tec files we want to plot (with paths from this notebook's working directory) and then create a list of tec files. This is to facilitate adding multiple tec files to plot side-by-side, but it will still work with only a single tec file in the "list".

In [ ]:
# Names of the tec files we want to plot
real_tec = './infiles/nitrogen_cycling_in_hanford_300_river_sediments_0d-obs-0.tec'
literature_tec = './infiles/nitrogen_cycling_0d-obs-0.tec'

# List all the files we want to iterate over
tecfiles = [real_tec, literature_tec]

## Organize data and clean column names

In this cell, we shorten the PFLOTRAN-assigned names to make them human-readable and organize them into a dictionary to make them easier to call later.

In [ ]:

results = {}

for i in tecfiles:
    df = pd.read_table(i, sep='\s+', header=None, engine='python', skiprows=[0])
    df_header = pd.read_table(i, sep=',', nrows=0)
    cols = list(df_header.columns.values)
    goodcols = [j.strip().strip('"').split('-',1)[-1].split('Free',1)[-1]
                .strip().replace(' ', '_').replace('_all', ' ').split(' ')[0] for j in cols]
    df.columns = goodcols
    unwanted = df.columns[df.columns.str.startswith('Total_')]
    dfclean = df.drop(unwanted, axis=1)
    results.update({i.split('/')[-1].split('-')[0]: dfclean})

## Sort into logical categories

The species_sorter function included in the tools script is designed to divide outputs into logical categories automatically. Not all models will contain all of these categories (so you may end up with some empty lists) and in some cases you may want to divide them more manually, but we've found this useful for quick vizualization. 

In [ ]:
Nspecies_lit, Cspecies_lit, Pspecies_lit, Sspecies_lit, ions_lit, solids_lit, physical_lit, biomass_lit = speciessorter(results['nitrogen_cycling_0d'],
                                                           timepoints = False)

Nspecies_river, Cspecies_river, Pspecies_river, Sspecies_river, ions_river, solids_river, physical_river, biomass_river = speciessorter(results['nitrogen_cycling_in_hanford_300_river_sediments_0d'],
                                                           timepoints = False)

## Plot Results - Nitrogen cycle

In [ ]:
# Establish labeling we want to use
x = 'Time_[d]'
lit = results['nitrogen_cycling_0d']
river = results['nitrogen_cycling_in_hanford_300_river_sediments_0d']

In [ ]:
# Plot nitrogen species
Nitrogen_fig = plt.figure(1)
ax1 = Nitrogen_fig.add_subplot(211)
ax2 = Nitrogen_fig.add_subplot(212)

lit.plot(x=x, y=Nspecies_lit, ax=ax1, sharex=True)
river.plot(x=x, y=Nspecies_river, ax=ax2, sharex=True)

ax1.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1e'))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1e'))

ax1.set_ylim(0, 9e-7)
ax2.set_ylim(0, 9e-7)
ax1.set_xlim(0,0.25)
ax2.set_xlim(0,0.25)

ax1.set_title("Literature-based")
ax2.set_title("Site-specific")

ax1.legend(loc = 'center right', bbox_to_anchor=(1.35,.5), ncol=1)
ax2.legend(loc = 'center right', bbox_to_anchor=(1.35,.5), ncol=1)

## Carbon utilization

Next, we'll plot the carbon sources. You'll notice that with the standard sorting, it is hard to see what is happening with many of the carbon sources in the site-specific model because there are so many (thus the legend below the figure instead of beside it). If you want to inspect more carefully, you can plot one (or several) compounds at a time instead of the whole group using, for example:
`river.plot(x=x, y='Acetate-_[M]', ax=ax4, sharex=True)`

where the keys used for the `y` argument are the species names used by PFLOTRAN.

In [ ]:
Carbon_fig = plt.figure(2)
ax3 = Carbon_fig.add_subplot(211)
ax4 = Carbon_fig.add_subplot(212)

lit.plot(x=x, y=Cspecies_lit, ax=ax3, sharex=True)
river.plot(x=x, y=Cspecies_river, ax=ax4, sharex=True)

ax3.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.3e'))
ax4.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.3e'))

ax3.set_xlim(0,0.25)
ax4.set_xlim(0,0.25)

ax3.set_title("Literature-based")
ax4.set_title("Site-specific")

ax3.legend(loc = 'center right', bbox_to_anchor=(1.35,.5), ncol=1)
ax4.legend(loc = 'center right', bbox_to_anchor=(1.35,-1), ncol=3)